# Module 3: Particle transport by mean velocity

We already defined and populated `U` and `V` fields which represent mean velocities averaged over 10 years. <br>
In this module we will use these velocity fields to transport particles across the globe

Let's include everything we have done in the previous modules (everything is contained in `module_02.jl`)
and check that our building blocks are there (grid, U and V)

In [21]:
include("module_03.jl")

@show grid;
@show U;
@show V;

grid = 1440×600×1 ImmersedBoundaryGrid{Float64, Periodic, Bounded, Bounded} on CPU with 4×4×4 halo:
├── immersed_boundary: GridFittedBoundary{Field{Center, Center, Center, Nothing, LatitudeLongitudeGrid{Float64, Periodic, Bounded, Bounded, OffsetArrays.OffsetVector{Float64, Vector{Float64}}, Float64, Float64, Float64, Float64, OffsetArrays.OffsetVector{Float64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}, OffsetArrays.OffsetVector{Float64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}, OffsetArrays.OffsetVector{Float64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}, CPU}, Tuple{Colon, Colon, Colon}, OffsetArrays.OffsetArray{Bool, 3, Array{Bool, 3}}, Bool, FieldBoundaryConditions{BoundaryCondition{Oceananigans.BoundaryConditions.Periodic, Nothing}, BoundaryCondition{Oceananigans.BoundaryConditions.Periodic, Nothing}, BoundaryCondition{Oceananigans.Boundar

### Defining particle initial positions

We want to 

In [44]:
λ₀ = -76.0
φ₀ = 35.0

degree_spread_λ = 5.0
degree_spread_φ = 5.0

n_particles = 50

λₚ = λ₀ .+ degree_spread_λ .* (rand(n_particles) .- 0.5);
φₚ = φ₀ .+ degree_spread_φ .* (rand(n_particles) .- 0.5);

zₚ = 0.5 .* ones(n_particles);

### Particle in Oceananigans

In [45]:
lagrangian_particles = LagrangianParticles(x=λₚ, y=φₚ, z=zₚ)

50 LagrangianParticles with eltype Particle:
├── 3 properties: (:x, :y, :z)
├── particle-wall restitution coefficient: 1.0
├── 0 tracked fields: ()
└── dynamics: no_dynamics

### Last building blocks: Model and Simulation

A model in Oceananigans includes all the physical phenomena we want to represent. <br>

In our case we only want to have particles transported by the velocity fields we previously defined.


In [46]:

model = HydrostaticFreeSurfaceModel(grid = grid, velocities = PrescribedVelocityFields(u = U, v = V),
                                    coriolis  = nothing,
                                    buoyancy  = nothing,
                                    closure   = nothing,
                                    tracers   = (),
                                    particles = lagrangian_particles)

HydrostaticFreeSurfaceModel{CPU, ImmersedBoundaryGrid}(time = 0 seconds, iteration = 0)
├── grid: 1440×600×1 ImmersedBoundaryGrid{Float64, Periodic, Bounded, Bounded} on CPU with 4×4×4 halo
├── timestepper: QuasiAdamsBashforth2TimeStepper
├── tracers: ()
├── closure: Nothing
├── buoyancy: Nothing
├── coriolis: Nothing
└── particles: 50 Lagrangian particles with 3 properties: (:x, :y, :z)

### Defining a simulation

The last step before running a model is to define a Simulation. <br>
A Simulation in Oceananigans specifies all the technical details of the simulation:
- What is the time step?
- When are we stopping the simulation?
- What data do we want to collect?

In [47]:
Δt = 6hours

@show simulation = Simulation(model, Δt = Δt, stop_time = 10years)

start_time = [time_ns()]

simulation.callbacks[:progress] = Callback(progress, IterationInterval(500));
save_interval = 10days;

simulation.output_writers[:surface_fields] = JLD2OutputWriter(model,  (; u = model.velocities.u, v = model.velocities.v, particles=model.particles,),
                            schedule = TimeInterval(save_interval),
                            filename = "output_particles",
                            overwrite_existing = true);


simulation = Simulation(model, Δt = Δt, stop_time = 10years) = Simulation of HydrostaticFreeSurfaceModel{CPU, ImmersedBoundaryGrid}(time = 0 seconds, iteration = 0)
├── Next time step: 6 hours
├── Elapsed wall time: 0 seconds
├── Wall time per iteration: NaN years
├── Stop time: 10 years
├── Stop iteration : Inf
├── Wall time limit: Inf
├── Callbacks: OrderedDict with 4 entries:
│   ├── stop_time_exceeded => Callback of stop_time_exceeded on IterationInterval(1)
│   ├── stop_iteration_exceeded => Callback of stop_iteration_exceeded on IterationInterval(1)
│   ├── wall_time_limit_exceeded => Callback of wall_time_limit_exceeded on IterationInterval(1)
│   └── nan_checker => Callback of NaNChecker for u on IterationInterval(100)
├── Output writers: OrderedDict with no entries
└── Diagnostics: OrderedDict with no entries


### Let's run our first simulation!

Every ingredient is complete, let's run!

In [48]:
# Let's goo!
run!(simulation)

┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations /Users/simonesilvestri/.julia/packages/Oceananigans/qvnQL/src/Simulations/run.jl:167
┌ Info: Time:    0 seconds, iteration: 0, wall time: 1.062 seconds
└ @ Main /Users/simonesilvestri/development/coessing2022-hub/julia-ocean-model/module_03.jl:12
┌ Info:     ... simulation initialization complete (135.193 ms)
└ @ Oceananigans.Simulations /Users/simonesilvestri/.julia/packages/Oceananigans/qvnQL/src/Simulations/run.jl:202
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations /Users/simonesilvestri/.julia/packages/Oceananigans/qvnQL/src/Simulations/run.jl:112
┌ Info:     ... initial time step complete (968.750 μs).
└ @ Oceananigans.Simulations /Users/simonesilvestri/.julia/packages/Oceananigans/qvnQL/src/Simulations/run.jl:119
┌ Info: Time:     125 days, iteration: 500, wall time: 907.915 ms
└ @ Main /Users/simonesilvestri/development/coessing2022-hub/julia-ocean-model/module_03.jl:12
┌ Info: Time:     250 days

### Visualize the output

I have already defined a visualization function for the output in "visualize_particles.jl". <br>
(If you want to take a look, go ahead and ask if there are questions)

In [ ]:
include("visualize_particles.jl")
visualize_results("output_particles")

In [ ]:
display_mp4("output_particles.mp4")

# Try it yourself!

Try changing the particle parameters and see where the particles end up
- initial position 
- spread
- number of particles

You should see particles accumulate in regions of "convergence"  <br>
Some interesting positions to try are:

***Southern Ocean*** <br>
λ₀, φ₀ = -120.0, -60.0

***Equatorial Pacific Ocean*** <br>
λ₀, φ₀ = -160.0, 0.0

***West Africa Coast*** <br>
λ₀, φ₀ = 55.2, 8.3

***Gulf stream (north Atlantic)*** <br>
λ₀, φ₀ = -76.0, 30.65